In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [34]:
df = pd.read_csv('/content/Last Annotated problems Aircraft Data - Blad1 (2).csv')

In [35]:
training_df = df[(df['LOCATION'].notnull())|(df['PART'].notnull())|(df['TGGEDPROBLEM'].notnull())]
train, test = train_test_split(training_df, test_size=0.2, random_state=42)
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [4]:
len(train)

460

In [36]:
import csv
import random
import spacy

def process_csv(input_file, skip_rows=0):
    spacy_training_data = []
    with open(input_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for index, row in enumerate(reader):
            if index < skip_rows:
                continue  # Skip the first `skip_rows` rows

            text = row['PROBLEM']
            entities = []
            for entity_type in ['LOCATION', 'PART', 'TGGEDPROBLEM']:
                entity = row[entity_type]
                if entity and entity != 'None':  # Check if entity is not empty or 'None'
                    start = text.find(entity)
                    end = start + len(entity)
                    entities.append((start, end, entity_type))
            if entities:  # Only add to training data if entities were found
                spacy_training_data.append((text, {"entities": entities}))
    return spacy_training_data

nlp = spacy.blank("en")

ner = nlp.add_pipe("ner")
ner.add_label("LOCATION")
ner.add_label("PART")
ner.add_label("TGGEDPROBLEM")

input_file = '/content/train.csv'

spacy_training_data = process_csv(input_file)
print(len(spacy_training_data), spacy_training_data[0])


460 ('L/H ENG #4 CYL BAFFLE CRACKED.', {'entities': [(0, 10, 'LOCATION'), (15, 21, 'PART'), (22, 29, 'TGGEDPROBLEM')]})


In [37]:
from spacy.training import Example
from spacy.util import minibatch

examples = []
for text, annot in spacy_training_data:
    doc = nlp.make_doc(text)
    entities = annot.get("entities")
    spans = []
    for start, end, label in entities:
        # if label is None or label == 'None':
        #     continue  # Skip entities with None label

        try:
            span = doc.char_span(start, end, label=label)
            if span is None:
                print(f"Failed to create span for '{text}' with label '{label}', skipping this annotation.")
            else:
                spans.append(span)
        except ValueError as e:
            print(f"Error processing '{text}' with label '{label}': {e}")

    if spans:
        try:
            examples.append(Example.from_dict(doc, {"entities": spans}))
        except ValueError as e:
            print(f"Ignoring annotation for '{text}' due to error: {e}")



Failed to create span for 'STUDENT COULDN'T START A/C.' with label 'PART', skipping this annotation.
Failed to create span for 'RIVET SHEARED @ R/H AFT BAFFLE.' with label 'LOCATION', skipping this annotation.
Failed to create span for 'L/H ENGINE #4 AFT BAFFL ECRACKED IN MULTIPLE PLACES.' with label 'TGGEDPROBLEM', skipping this annotation.
Failed to create span for 'SCREW IS MISSING ON #3 CYL BAFFLE.' with label 'TGGEDPROBLEM', skipping this annotation.
Failed to create span for 'SPIRAL WRAP ON ROCKER OIL RETURN LINES #1 & 3 REQUIRES REPLA' with label 'PART', skipping this annotation.
Failed to create span for 'ALL SCREWS FOR ROCKER COVERS ARE LOOSE.' with label 'PART', skipping this annotation.
Failed to create span for '#3 ROCKER COVER REQUIRES NEW GASKET.' with label 'TGGEDPROBLEM', skipping this annotation.
Failed to create span for 'ALL ROCKER COVERS HAVE LOOSE SCREWS.' with label 'PART', skipping this annotation.
Failed to create span for 'AT 5500 FT, OAT +7, EXECUTED ENG FAILU

In [38]:
nlp.begin_training()
for epoch in range(32):
    random.shuffle(examples)
    for batch in minibatch(examples, size=8):
        nlp.update(batch)

# Save the trained model to a directory
output_dir = "ner_model"
nlp.to_disk(output_dir)
print("Trained NER model saved to:", output_dir)


Trained NER model saved to: ner_model


In [39]:
# Load the trained model from the directory
loaded_nlp = spacy.load(output_dir)

spacy_test_data = process_csv('/content/test.csv')
# print(len(spacy_test_data))

test_examples = []
for text, annot in spacy_test_data:
    doc = nlp.make_doc(text)
    entities = annot.get("entities")
    spans = []
    for start, end, label in entities:
        # if label is None or label == 'None':
        #     continue  # Skip entities with None label

        try:
            span = doc.char_span(start, end, label=label)
            if span is None:
                print(f"Failed to create span for '{text}' with label '{label}', skipping this annotation.")
            else:
                spans.append(span)
        except ValueError as e:
            print(f"Error processing '{text}' with label '{label}': {e}")

    if spans:
        try:
            test_examples.append(Example.from_dict(doc, {"entities": spans}))
        except ValueError as e:
            print(f"Ignoring annotation for '{text}' due to error: {e}")

print(len(test_examples))



Failed to create span for '#2 INDUCTION TUBE HOSE CLAMP BROKEN.' with label 'PART', skipping this annotation.
Failed to create span for 'CAP SCREW MISSING, L/H ENG #4 BAFFLE.' with label 'TGGEDPROBLEM', skipping this annotation.
Failed to create span for '#4 CYLINDER COMPRESSION @ 45/80.' with label 'TGGEDPROBLEM', skipping this annotation.
Failed to create span for 'RIVET PULLED THROUGH BAFFLE SEAL, #1 CYL.' with label 'TGGEDPROBLEM', skipping this annotation.
115


In [40]:

# Evaluate the performance of the model
scores = loaded_nlp.evaluate(test_examples)
print("Evaluation scores:", scores)

Evaluation scores: {'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.8345070422535211, 'ents_r': 0.8434163701067615, 'ents_f': 0.8389380530973451, 'ents_per_type': {'LOCATION': {'p': 0.8785046728971962, 'r': 0.912621359223301, 'f': 0.8952380952380954}, 'TGGEDPROBLEM': {'p': 0.8118811881188119, 'r': 0.7884615384615384, 'f': 0.8}, 'PART': {'p': 0.8026315789473685, 'r': 0.8243243243243243, 'f': 0.8133333333333335}}, 'speed': 14270.654005310487}


In [41]:
print(df.columns)


Index(['IDENT', 'PROBLEM', 'LOCATION', 'PART', 'TGGEDPROBLEM', 'ACTION'], dtype='object')


In [43]:
# Filter the dataframe for rows where LOCATION, PART, and TAGGEDPROBLEM are null
raw_df = df[(df['LOCATION'].isnull())&(df['PART'].isnull())&(df['TGGEDPROBLEM'].isnull())]

# Create new columns for LOCATION, PART, and TAGGEDPROBLEM based on the predicted entities
raw_df['Location'] = raw_df['PROBLEM'].apply(lambda prb: ', '.join([ent.text for ent in loaded_nlp(prb).ents if ent.label_ == 'LOCATION']))
raw_df['Part'] = raw_df['PROBLEM'].apply(lambda prb: ', '.join([ent.text for ent in loaded_nlp(prb).ents if ent.label_ == 'PART']))
raw_df['Tagged_problem'] = raw_df['PROBLEM'].apply(lambda prb: ', '.join([ent.text for ent in loaded_nlp(prb).ents if ent.label_ == 'TGGEDPROBLEM']))

# Now, you have separate columns for each label
raw_df[['Location', 'Part', 'Tagged_problem']]

<ipython-input-43-1988976210ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['Location'] = raw_df['PROBLEM'].apply(lambda prb: ', '.join([ent.text for ent in loaded_nlp(prb).ents if ent.label_ == 'LOCATION']))
<ipython-input-43-1988976210ab>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['Part'] = raw_df['PROBLEM'].apply(lambda prb: ', '.join([ent.text for ent in loaded_nlp(prb).ents if ent.label_ == 'PART']))
<ipython-input-43-1988976210ab>:7: SettingWithCopyWarning: 
A value is trying 

,Location,Part,Tagged_problem
287,,,LOWERED
499,,"AFT L/H, BAFFLE",CRACKED
500,L/H,BAFFLE,CRACKED
501,ALL 4,ROCKER COVER GASKETS,LEAKING
502,#2 CYL,INTAKE GASKET,LEAKING
...,...,...,...
6162,,BAFFLE PLUGS,WORN
6163,,,FLUCTUATING
6164,#2,INTAKE,LEAKING
6165,L/H FWD ENGINE,BAFFLE,CRACKED


In [44]:
import pandas as pd

# Assuming `df` is the original dataframe
# And `raw_df` contains the predictions

# Update the LOCATION column with predicted labels where original labels are missing
df['LOCATION'] = df['LOCATION'].combine_first(raw_df['Location'])

# Update the PART column with predicted labels where original labels are missing
df['PART'] = df['PART'].combine_first(raw_df['Part'])

# Update the TGGEDPROBLEM column with predicted labels where original labels are missing
df['TGGEDPROBLEM'] = df['TGGEDPROBLEM'].combine_first(raw_df['Tagged_problem'])

In [45]:
df = df.drop(columns=['PROBLEM', 'ACTION'])

# Save the updated dataframe to a CSV file
df.to_csv('Problem_extraction_NER.csv', index=False)

print("Final labels saved to 'Problem_extraction_NER.csv'")

Final labels saved to 'Problem_extraction_NER.csv'
